# Autoencoders using HE --> IHC


In [1]:
pwd


'C:\\Users\\Sidra\\Downloads'

In [2]:
pip install scikit-image

Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from torch.autograd import Variable
import torchvision
from torchvision import datasets
import matplotlib.pyplot as plt
import os
import cv2
import numpy as np
import torch.nn.functional as F
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder

Matplotlib is building the font cache; this may take a moment.


ModuleNotFoundError: No module named 'cv2'

In [ ]:
# Define the autoencoder model
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        # Encoder layers
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # Decoder layers
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 3, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
# Set device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# Hyperparameters
num_epochs = 100
batch_size = 64
learning_rate = 0.005

# Image transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

# Load the dataset
he_set = torchvision.datasets.ImageFolder('C:/Users/sidra/Desktop/bci/he/train', transform=transform)
he_loader = torch.utils.data.DataLoader(he_set , batch_size=batch_size, shuffle=True)


In [ ]:
# Initialize the model

model = Autoencoder().to(device)

In [ ]:
# Set loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(num_epochs):
    running_loss = 0.0
    
    for he_images, _ in he_loader:
        # Move data to the selected device
        he_images = he_images.to(device)
        
        # Forward pass
        outputs = model(he_images)
        
        # Compute loss
        loss = criterion(outputs, he_images)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    # Print the average loss for the epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {loss.item():.4f}")

In [ ]:
# Save the trained model
#torch.save(model, 'autoencode_model.pt')

In [ ]:
image1 = image.open("C:/Users/sidra/Desktop/bci/he/new/00000_train_1+.png")

In [ ]:
image2=Image.open("C:/Users/sidra/Desktop/bci/ihc/new/00000_train_1+.png")

In [ ]:
image1

In [ ]:
image2

In [ ]:
import PIL.Image as Image
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [ ]:
def psnr(image1, image2, max_value=1.0):
    mse = torch.mean((image1 - image2) ** 2)
    psnr_value = 20 * torch.log10(max_value / torch.sqrt(mse))
    return psnr_value

def ssim(image1, image2, window_size=20, sigma=1.5, data_range=1.0, k1=0.01, k2=0.03):
    mu1 = F.avg_pool2d(image1, window_size, stride=1, padding=window_size//2)
    mu2 = F.avg_pool2d(image2, window_size, stride=1, padding=window_size//2)

    sigma1_sq = F.avg_pool2d(image1 ** 2, window_size, stride=1, padding=window_size//2) - mu1 ** 2
    sigma2_sq = F.avg_pool2d(image2 ** 2, window_size, stride=1, padding=window_size//2) - mu2 ** 2
    sigma12 = F.avg_pool2d(image1 * image2, window_size, stride=1, padding=window_size//2) - mu1 * mu2

    c1 = (k1 * data_range) ** 2
    c2 = (k2 * data_range) ** 2

    ssim_value = ((2 * mu1 * mu2 + c1) * (2 * sigma12 + c2)) / ((mu1 ** 2 + mu2 ** 2 + c1) * (sigma1_sq + sigma2_sq + c2))
    ssim_value = torch.mean(ssim_value)

    return ssim_value

In [ ]:
import matplotlib.pyplot as plt
import glob

# Define the path to the directory containing the images
img_dir = "C:/Users/sidra/Desktop/bci/he/test/*.png"
#glob=global
# Use glob to get all the image file names in the directory
#We can use the function glob.glob() directly from glob module to retrieve paths recursively from 
#inside the directories/files and subdirectories/subfiles.
img_files = glob.glob(img_dir)

# Loop over all the image files and display them using matplotlib
for img_file in img_files:
    # Read the image using matplotlib.imread() function
    img = plt.imread(img_file)
    
# Display the image using matplotlib.pyplot.imshow() function
    plt.title("HE Images")
    plt.imshow(img)
    plt.show()

In [ ]:
## Load the trained model
# model = Autoencoder().to(device)
# model.load_state_dict(torch.load('autoencoder_model.pth'))

In [ ]:
model.eval()

# Test the autoencoder using H&E test images and  generate IHC images
test_dataset = torchvision.datasets.ImageFolder('C:/Users/sidra/Desktop/bci/he/test', transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

total_psnr = 0.0
total_ssim = 0.0
num_samples = 0


for i,(he_image, _) in enumerate(test_loader):
    # Move data to the selected device
    he_image = he_image.to(device)
    
    # Perform image translation
    ihc_image = model(he_image)
   
    
    # Calculate PSNR and SSIM
    psnr_value = psnr(ihc_image, he_image)
    ssim_value = ssim(ihc_image, he_image)

    
    
    
    total_psnr += psnr_value.item()
    total_ssim += ssim_value.item()
    num_samples += 1
    
    
    # Convert tensor to numpy array and save the generated IHC image
    ihc_image = ihc_image.detach().cpu().numpy()
#     # Save the generated IHC image
#     ihc_image.save(ihc_image, 'C:/Users/PG LAB/Desktop/bci ds/full/all')
# Convert numpy array to PIL image
#     ihc_image = Image.fromarray((ihc_image * 255).astype(np.uint8).squeeze())
    ihc_image = (ihc_image[0].transpose(1, 2, 0) + 1) / 2.0  # Convert from [-1, 1] to [0, 1]

    # Scale the values to [0, 255]
    ihc_image = (ihc_image * 255).astype(np.uint8)

    # Convert numpy array to PIL image
    ihc_image = Image.fromarray(ihc_image)

    # Save the generated IHC image
    ihc_image.save(f'C:/Users/sidra/Desktop/bci/newimg.png')
    
print("Autoencoder Image-to-image translation complete.")
average_psnr = total_psnr / num_samples
average_ssim = total_ssim / num_samples
print("Average PSNR:{:.4f}".format(average_psnr))
print("Average SSIM:{:.4f}".format(average_ssim))


In [ ]:
import torchvision.transforms as T

In [ ]:
import torch
from torchvision.transforms import ToPILImage


In [ ]:
# Convert the tensor to a PIL image
pil= tensor_to_pil(he_image)

In [ ]:
he_image.show()  # Display the image
pil_image.save("image1.png")  # Save the image to a file

In [ ]:
ihc_image

In [ ]:
psnr_values = []  # List to store PSNR values
ssim_values = []  # List to store SSIM values

for i,(he_image, _) in enumerate(test_loader):
    # Move data to the selected device
    he_image = he_image.to(device)
    
    # Perform image translation
    ihc_image = model(he_image)
    
    # Calculate PSNR and SSIM
    psnr_value = psnr(ihc_image, he_image)
    ssim_value = ssim(ihc_image, he_image)

    
    # Append PSNR and SSIM values to the respective lists
    psnr_values.append(psnr_value.item())
    ssim_values.append(ssim_value.item())

# Convert the lists to NumPy arrays
psnr_values = np.array(psnr_values)
ssim_values = np.array(ssim_values)

# Generate x-axis values
x = np.arange(1, num_samples + 1)

# Plot the PSNR values
plt.figure(figsize=(12, 6))
plt.plot(x, psnr_values, label='PSNR')
plt.xlabel('Image')
plt.ylabel('PSNR')
plt.title('PSNR values')
plt.legend()
plt.grid(True)
plt.show()

# Plot the SSIM values
plt.figure(figsize=(12, 6))
plt.plot(x, ssim_values, label='SSIM')
plt.xlabel('Image')
plt.ylabel('SSIM')
plt.title('SSIM values')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
torch.save(model,'autoencodernew_model.pt')


In [ ]:
model = torch.load('autoencodernew_model.pt')


In [ ]:
model.eval()